In [ ]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia analizę algorytmu Symulowanego Wyżarzania (SA)
# dla różnych zestawów parametrów. Dla każdej konfiguracji
# wykonywanych jest kilka powtórzeń, zapisywany jest koszt,
# czas wykonania oraz średnie statystyki. Wyniki trafiają
# do pliku CSV przy użyciu save_experiment_results().

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.sa_numba import solve_tsp
from src.utils.result_saver import save_experiment_results


# USTAWIENIA
TSP_FILES = ["Dane_TSP_76.xlsx"]

PARAM_GRID = {
    "T0": [5000],
    "alpha": [0.999999],
    "T_min": [0.0001],
    "max_iter": [10_000_000],
    "neighborhood_type": ["two_opt"],
}

REPEATS = 6
results = []


# ROZGRZANIE NUMBA (KOMPILACJA JIT)
print("Rozgrzewanie Numba (kompilacja JIT)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"T0": 1000, "max_iter": 100, "neighborhood_type": "swap"})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for T0 in PARAM_GRID["T0"]:
        for alpha in PARAM_GRID["alpha"]:
            for T_min in PARAM_GRID["T_min"]:
                for max_iter in PARAM_GRID["max_iter"]:
                    for neighborhood_type in PARAM_GRID["neighborhood_type"]:

                        costs = []
                        runtimes = []

                        for _ in range(REPEATS):
                            params = {
                                "T0": T0,
                                "alpha": alpha,
                                "T_min": T_min,
                                "max_iter": max_iter,
                                "neighborhood_type": neighborhood_type,
                            }

                            start_t = time.perf_counter()
                            route, cost, runtime, meta = solve_tsp(D, params)
                            total_time = time.perf_counter() - start_t

                            costs.append(cost)
                            runtimes.append(total_time)

                        results.append({
                            "instance": tsp_file,
                            "T0": T0,
                            "alpha": alpha,
                            "T_min": T_min,
                            "max_iter": max_iter,
                            "neighborhood_type": neighborhood_type,
                            "mean_cost": round(np.mean(costs), 3),
                            "min_cost": round(np.min(costs), 3),
                            "mean_runtime": np.mean(runtimes),
                        })


# ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

df = pd.DataFrame(results)
save_experiment_results(df, time_seconds=int(elapsed), subfolder="SA")


# RAPORT
print("\nPodsumowanie wyników:")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]]
      .mean()
      .round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\nNajlepsze parametry:\n{best.to_dict()}")

Warm-up Numba (kompilacja JIT)...
Kompilacja zakończona. Start eksperymentu...


Instancja: Dane_TSP_76.xlsx

Podsumowanie (pierwsze 20 wierszy):
        instance   T0    alpha  T_min  max_iter neighborhood_type  mean_cost  min_cost  mean_runtime
Dane_TSP_76.xlsx 5000 0.999999 0.0001  10000000           two_opt  99192.872 97919.087      4.681982

Podsumowanie wyników:
                   mean_cost   min_cost  mean_runtime
neighborhood_type                                    
two_opt            99192.872  97919.087         4.682

Najlepsze parametry: {'instance': 'Dane_TSP_76.xlsx', 'T0': 5000, 'alpha': 0.999999, 'T_min': 0.0001, 'max_iter': 10000000, 'neighborhood_type': 'two_opt', 'mean_cost': 99192.872, 'min_cost': 97919.087, 'mean_runtime': 4.681982425200113}
